<h1> Datto RMM - Reports Summary </h1>

In [ ]:
# general purpose
import time
import datetime as dt


# Web Bot
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# Data Ingest and Export
import pandas as pd
import re
import json

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# Create Bot Object

## Create Firefox Driver

In [ ]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

firefox_dev_binary = FirefoxBinary(r'C:/Program Files/Firefox Developer Edition/firefox.exe')
f_driver = webdriver.Firefox(firefox_binary=firefox_dev_binary, executable_path=r'D:/web_drivers/mozilla/geckodriver-v0.31.0-win64/geckodriver.exe')

## Create Chrome Driver

## Request Header Info (if needed to impersonate api browser calls)

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://concord.centrastage.net'
auth_url = f'https://auth.datto.com'
#home_url = f'{baseUrl}'
links = []

# Login to DattoRMM

In [ ]:
f_driver.get(auth_url)

In [ ]:
title = f_driver.find_element_by_xpath('//h1')
title.text

In [ ]:
login_txtbox = f_driver.find_element_by_xpath("//input[@id='form_username']")
login_txtbox.send_keys((config['dattormm']['username']),Keys.ENTER)

In [ ]:
pass_txtbox = f_driver.find_element_by_xpath("//input[@id='form_password']")
pass_txtbox.send_keys((config['dattormm']['password']),Keys.ENTER)

In [ ]:
time.sleep(3)

In [ ]:
mfa_txtbox = f_driver.find_element_by_id('authy_token')
mfa_txtbox.send_keys('304203', Keys.ENTER) # number entered manually for now /TODO: automate MFA

# Get Active Reports Summary

In [ ]:
active_reports_url = 'https://concord.centrastage.net/csm/report/reports?mode=active'

In [ ]:
f_driver.get(active_reports_url)

In [ ]:
table = f_driver.find_element_by_xpath("//table[@class='list']")

In [ ]:
header_list = ['reportName','includedReports','reportType','schedule','reportLanguage','nextRunTime','lastRunTime','Status']

In [ ]:
table_contents = table.find_elements_by_tag_name('td')

In [ ]:
contents_list = []
for index, content in enumerate(table_contents):
    content = content.text
    if content != '':
        contents_list.append(content)

## Cycling through headers to create list of dictionaries.  A dictionary key pair per content line and another per row, resetting the dict when the index matches the len of the header list

In [ ]:
i = 0
print(f'Headers list len is: {len(header_list)}')
report_dict = {}
table_list = []
for index, content in enumerate(contents_list):
    content_dict = {}
    if re.search('\n', content):
        content_dict[header_list[i]] = re.findall('([^\n]+)+\n?',content)
    else:
        content_dict[header_list[i]] = content
    report_dict.update(content_dict)
    if i >= len(header_list)-1:
        i = 0
        table_list.append(report_dict)
        report_dict = {}
    else:
        i = i+1

In [ ]:
df_active_reports = pd.DataFrame(table_list)

In [ ]:
df_active_reports.to_csv('[REDACTED]/.csv')

In [ ]:
df_active_reports

### Import Modules, Create env Variables

In [ ]:
# import and assign secrets from env.ini
base_uri = config['dattormm']['base_uri']
api_key = config['dattormm']['api_key']
api_secret = config['dattormm']['api_secret']
# call token api url
token_uri = f'{base_uri}/auth/oauth/token'

## Create DataFrame via API Call Iteration


In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = api_key
data['password'] = api_secret

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']
## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/api/v2/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)

In [ ]:
client_list = list(df_devices['siteName'].unique())

In [ ]:
client_list

In [ ]:
report_clients_list = []

for name in df_active_reports['reportName']:
    try:
        split_name = re.search('Remote\sActivity\s\-\s([^\n]+)',name).group(1)
        #print(split_name)
        report_clients_list.append(split_name)
    except:
        continue

In [ ]:
report_clients_list

In [ ]:
for client in client_list:
    if client not in report_clients_list:
        print(f'Datto siteName [{client}] not in report_clients_list')
    else:
        continue

In [ ]:
df_active_reports['reportName']